# 2. 调频（FM - Frequency Modulation）

**原理**：调频是通过改变载波的**频率**来表示基带信号的信息。载波的瞬时频率随基带信号的幅度变化而变化，通常用于音频广播和移动通信。

公式：
$$s(t) = \cos\left(2 \pi f_c t + k_f \int m(\tau) d\tau\right)$$
其中：
- $k_f$是调频灵敏度常数，
- $\int m(\tau) d\tau$ 是基带信号的积分，用于控制频率变化。

## 2.1. 调频代码演示

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

# 自定义 cumtrapz 函数
def cumtrapz_custom(y, x=None, dx=1.0):
    y = np.asarray(y)
    if x is None:
        dx_array = dx
    else:
        x = np.asarray(x)
        dx_array = np.diff(x)
    cumulative_integral = np.cumsum((y[:-1] + y[1:]) / 2 * dx_array)
    return np.concatenate(([0], cumulative_integral))

# 定义参数
fc = 1000  # 载波频率 (Hz)
fs = 10000  # 采样频率 (Hz)
t = np.arange(0, 0.01, 1/fs)  # 时间轴

# 基带信号 (低频信号)
message_signal = np.sin(2 * np.pi * 100 * t)  # 100 Hz的基带信号

# 调频灵敏度
kf = 50  # 增大调频灵敏度系数

# 使用自定义的cumtrapz函数进行积分
message_integral = cumtrapz_custom(message_signal, x=t)

# 调频信号
modulated_signal_fm = np.cos(2 * np.pi * fc * t + kf * message_integral)

# 解调：相位差分
demodulated_signal_fm = np.diff(np.unwrap(np.angle(modulated_signal_fm))) * (fs / (2 * np.pi * kf))

# 去除解调信号的直流偏移
demodulated_signal_fm -= np.mean(demodulated_signal_fm)

# 使用Butterworth低通滤波器进行滤波，截止频率为 80 Hz
b, a = butter(4, 80 / (fs / 2), btype='low')
demodulated_signal_fm = filtfilt(b, a, demodulated_signal_fm)

# 调整时间轴长度
t_valid = t[:len(demodulated_signal_fm)]

# 绘制结果
plt.figure(figsize=(10, 6))
plt.subplot(3, 1, 1)
plt.plot(t, message_signal)
plt.title("Message Signal (Original Baseband)")

plt.subplot(3, 1, 2)
plt.plot(t, modulated_signal_fm)
plt.title("FM Modulated Signal")

plt.subplot(3, 1, 3)
plt.plot(t_valid, demodulated_signal_fm)
plt.title("FM Demodulated Signal (Recovered Baseband)")

plt.tight_layout()
plt.show()

# 调整解调信号长度与原始信号一致
demodulated_signal_fm_resized = np.resize(demodulated_signal_fm, len(message_signal))

# 1. 计算均方误差 (MSE)
mse = mean_squared_error(message_signal, demodulated_signal_fm_resized)
print("Mean Squared Error (MSE):", mse)

# 2. 计算相关系数
correlation_coefficient, _ = pearsonr(message_signal, demodulated_signal_fm_resized)
print("Correlation Coefficient:", correlation_coefficient)

# 3. 绘制频谱对比
plt.figure(figsize=(12, 6))
# 原始信号的频谱
plt.subplot(2, 1, 1)
plt.magnitude_spectrum(message_signal, Fs=fs, scale='dB')
plt.title("Spectrum of Original Message Signal")

# 解调信号的频谱
plt.subplot(2, 1, 2)
plt.magnitude_spectrum(demodulated_signal_fm_resized, Fs=fs, scale='dB')
plt.title("Spectrum of Demodulated Signal")

plt.tight_layout()
plt.show()


## 2.2. 调频解调（FM Demodulation）

**原理**：调频解调是通过检测**载波瞬时频率的变化**来提取基带信号。常用的FM解调方法包括**差分解调**，即对相邻信号进行相位差计算，从而恢复出频率变化。

**步骤**：
1. 对调制信号进行微分，获得瞬时频率的变化。
2. 使用低通滤波器去除高频成分，保留基带信号。

## 2.3. 代码演示

In [ ]:
from scipy.signal import butter, filtfilt

# 调整调频灵敏度
kf = 25  # 降低调频灵敏度系数

# 重新生成调频信号
modulated_signal_fm = np.cos(2 * np.pi * fc * t + kf * message_integral)

# 差分解调（计算相邻样本的差）
demodulated_signal_fm = np.diff(np.unwrap(np.angle(modulated_signal_fm)))

# 使用Butterworth低通滤波器进行滤波
b, a = butter(4, 150 / (fs / 2), btype='low')  # 调整截止频率
demodulated_signal_fm = filtfilt(b, a, demodulated_signal_fm)

# 调整时间轴长度
t_valid = t[:len(demodulated_signal_fm)]

# 绘图
plt.figure(figsize=(10, 6))
plt.plot(t_valid, demodulated_signal_fm)
plt.title("FM Demodulated Signal (Recovered Baseband with Adjusted kf and Filter)")

plt.tight_layout()
plt.show()


### 总结
- **AM**：通过改变幅度来调制。
- **FM**：通过改变频率来调制。
- **PM**：通过改变相位来调制。

不同的调制方式在不同的应用中各有优势，根据传输距离、带宽需求和抗干扰能力来选择合适的方式。


- **AM解调**：通过包络检波（整流 + 低通滤波）来恢复基带信号。
- **FM解调**：通过差分相位检测来提取频率变化，再通过低通滤波提取基带信号。
- **PM解调**：同样利用相位差分来还原基带信号，适用于相位调制的信号。

这些解调方法各有优势，根据信号类型和需求选择合适的解调方案。